## Problem Statement

We have been given a PDE: u(t,x) = du/dt + u * du/dx - (0.01/pi)*d2u/d2x
and boundary condition: u(0,x) = -sin(pi*x) ; u(t,1)=u(t,-1)=0 ; x E [-1, 1] ; t E [0,1]

- Independent variables: x,t (input)
- Dependent variables: u (outputs)


We have to find out u(x,t) for all x in range [-1,1] and t in range [0,1]

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
%matplotlib inline
import time
import copy
#plt.style.use('dark_background')

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden_layer1 = nn.Linear(2,20)
        self.hidden_layer2 = nn.Linear(20,20)
        self.hidden_layer3 = nn.Linear(20,20)
        self.hidden_layer4 = nn.Linear(20,20)
        self.hidden_layer5 = nn.Linear(20,20)
        self.hidden_layer6 = nn.Linear(20,20)
        self.hidden_layer7 = nn.Linear(20,20)
        self.hidden_layer8 = nn.Linear(20,20)
        self.output_layer = nn.Linear(20,1)

    def forward(self, t,x):
        inputs = torch.cat([x,t],axis=1)
        layer1_out = torch.sigmoid(self.hidden_layer1(inputs))
        layer2_out = torch.sigmoid(self.hidden_layer2(layer1_out))
        layer3_out = torch.sigmoid(self.hidden_layer3(layer2_out))
        layer4_out = torch.sigmoid(self.hidden_layer4(layer3_out))
        layer5_out = torch.sigmoid(self.hidden_layer5(layer4_out))
        layer6_out = torch.sigmoid(self.hidden_layer6(layer5_out))
        layer7_out = torch.sigmoid(self.hidden_layer7(layer6_out))
        layer8_out = torch.sigmoid(self.hidden_layer8(layer7_out))
        output = self.output_layer(layer8_out)
        return output
    
net = Net()
net = net.to(device)

In [3]:
def f(t,x, net):
    u = net(t,x)
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]
    u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
    pde = u_t + u*u_x - (0.01/np.pi)*u_xx
    return pde

mse_cost_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters())

In [4]:
bc_size = 2000
pt_size = 2000

x_bc = np.random.uniform(low=-1.0, high=1.0, size=(bc_size,1))
x_bc[0][0] = -1.0
x_bc[-1][0] = 1.0
t_bc = np.random.uniform(low=0.0, high=1.0, size=(bc_size,1))
t_bc[0][0] = 0.0
t_bc[-1][0] = 1.0
u_bc = -np.sin(np.pi*x_bc)

bc_x = Variable(torch.from_numpy(x_bc).float(), requires_grad=False).to(device)
bc_t = Variable(torch.from_numpy(t_bc).float(), requires_grad=False).to(device)
bc_u = Variable(torch.from_numpy(u_bc).float(), requires_grad=False).to(device)

zeros_bc = np.zeros((bc_size,1))
ones_bc = np.ones((bc_size,1))
neg_ones_bc = -1.*np.zeros((bc_size,1))

bc_all_zeros = Variable(torch.from_numpy(zeros_bc).float(), requires_grad=False).to(device)
bc_all_ones = Variable(torch.from_numpy(ones_bc).float(), requires_grad=False).to(device)
bc_all_neg_ones = Variable(torch.from_numpy(neg_ones_bc).float(), requires_grad=False).to(device)
    
zeros_pt = np.zeros((pt_size,1))   
pt_all_zeros = Variable(torch.from_numpy(zeros_pt).float(), requires_grad=False).to(device)

In [ ]:
iterations = 200000
lowest_loss = 99999999.0
losses = []
start = time.time()


for epoch in range(iterations):
    optimizer.zero_grad()
    
    
    # Loss based on boundary condition (-sin(pi*x))
    net_bc_out = net(bc_all_zeros, bc_x)
    mse_u = mse_cost_function(net_bc_out, bc_u)
    
    
    # Loss based on boundary condition (x=1)
    net_bc_out = net(bc_t, bc_all_ones)
    mse_one = mse_cost_function(net_bc_out, bc_all_zeros)
    
    
    # Loss based on boundary condition (x=-1)
    net_bc_out = net(bc_t, bc_all_neg_ones)
    mse_none = mse_cost_function(net_bc_out, bc_all_zeros)
    
    
    # Loss based on PDE
    x_pt = np.random.uniform(low=-1.0, high=1.0, size=(pt_size,1))
    t_pt = np.random.uniform(low=0.0, high=1.0, size=(pt_size,1))
    pt_x_collocation = Variable(torch.from_numpy(x_pt).float(), requires_grad=True).to(device)
    pt_t_collocation = Variable(torch.from_numpy(t_pt).float(), requires_grad=True).to(device)
    
    f_out = f(pt_t_collocation, pt_x_collocation, net)
    mse_f = mse_cost_function(f_out, pt_all_zeros)
    
    
    # Combining the loss functions
    loss = mse_u + mse_f + mse_one + mse_none
    

    with torch.autograd.no_grad():
        if loss.data < lowest_loss:
            lowest_loss = loss.data
            best_copy = copy.deepcopy(net)
        losses.append(lowest_loss)
        #if lowest_loss <= .0001:
        #    break;
        if (epoch+1)%1000 == 0:
            print(epoch+1,"Training Loss:",loss.data)
            
    loss.backward() # This is for computing gradients using backward propagation
    optimizer.step() # This is equivalent to : theta_new = theta_old - alpha * derivative of J w.r.t theta
    
    
# Save Model
torch.save(best_copy.state_dict(), "burger_model.pt")
print('Completed',epoch+1,'iterations in',round((time.time()-start)/60,0), 'minutes')
print('Model Loss:',lowest_loss,'\n')

xs = list(range(len(losses)))
ys = np.zeros((len(losses),1))
plt.plot(xs,ys)
plt.plot(xs,losses)
plt.show()

1000 Training Loss: tensor(0.2430)
2000 Training Loss: tensor(0.2388)
3000 Training Loss: tensor(0.2367)
4000 Training Loss: tensor(0.2372)
5000 Training Loss: tensor(0.2333)
6000 Training Loss: tensor(0.1036)
7000 Training Loss: tensor(0.0967)
8000 Training Loss: tensor(0.0905)
9000 Training Loss: tensor(0.0826)
10000 Training Loss: tensor(0.0800)
11000 Training Loss: tensor(0.0757)
12000 Training Loss: tensor(0.0733)
13000 Training Loss: tensor(0.0707)
14000 Training Loss: tensor(0.0693)
15000 Training Loss: tensor(0.0683)
16000 Training Loss: tensor(0.0674)
17000 Training Loss: tensor(0.0663)
18000 Training Loss: tensor(0.0638)
19000 Training Loss: tensor(0.0612)
20000 Training Loss: tensor(0.0564)
21000 Training Loss: tensor(0.0517)
22000 Training Loss: tensor(0.0478)
23000 Training Loss: tensor(0.0460)
24000 Training Loss: tensor(0.0449)
25000 Training Loss: tensor(0.0440)
26000 Training Loss: tensor(0.0427)
27000 Training Loss: tensor(0.0428)
28000 Training Loss: tensor(0.0430)
2

In [ ]:
net = Net()
net.load_state_dict(torch.load('burger_model.pt'))

def plot_3D_map(net):
    fig = plt.figure(figsize = (15,10))
    ax = fig.gca(projection='3d')
    fig.suptitle('3D Model Distribution')

    x=np.arange(-1,1,0.01)
    t=np.arange(0,1,0.01)
    ms_x, ms_t = np.meshgrid(x, t)

    x = np.ravel(ms_x).reshape(-1,1)
    t = np.ravel(ms_t).reshape(-1,1)

    pt_x = Variable(torch.from_numpy(x).float(), requires_grad=True).to(device)
    pt_t = Variable(torch.from_numpy(t).float(), requires_grad=True).to(device)
    pt_u = net(pt_t,pt_x)
    u=pt_u.data.cpu().numpy()
    ms_u = u.reshape(ms_x.shape)
    surf = ax.plot_surface(ms_x,ms_t,ms_u, cmap=cm.coolwarm,linewidth=0, antialiased=False)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    fig.colorbar(surf, shrink=0.5, aspect=5)

    plt.show()
    
def plot_heat_map(net):
    x=np.arange(-1,1,0.01)
    t=np.arange(0,1,0.01)
    ms_x, ms_t = np.meshgrid(x, t)

    xx = np.ravel(ms_x).reshape(-1,1)
    tt = np.ravel(ms_t).reshape(-1,1)
    pt_x = Variable(torch.from_numpy(xx).float(), requires_grad=True).to(device)
    pt_t = Variable(torch.from_numpy(tt).float(), requires_grad=True).to(device)
    pt_u = net(pt_t,pt_x)
    
    u=pt_u.data.cpu().numpy()
    ms_u = u.reshape(ms_x.shape)
    
    fig = plt.figure(figsize=(10,5))
    plt.imshow(u, cmap='rainbow_r', interpolation='nearest', aspect=.25)
    plt.title("2D Model Distribution")
    plt.colorbar()
    plt.show()
    
def plot_t_value(t_value, net):
    x=np.arange(-1,1,.01)
    xx=np.ravel(x).reshape(-1,1)
    t = np.ones((len(x),1))*t_value
    
    pt_x = Variable(torch.from_numpy(xx).float(), requires_grad=True).to(device)
    pt_t = Variable(torch.from_numpy(t).float(), requires_grad=True).to(device)
    pt_u = net(pt_t,pt_x)
    
    u=pt_u.data.cpu().numpy()
    
    if t_value == 0.0:
        ys = -np.sin(np.pi*x)
        plt.plot(x,ys,'r')
    plt.plot(x,u,'cyan')
    plt.title('Model at t='+str(t_value))
    plt.show()
    
plot_3D_map(net)
plot_heat_map(net)
plot_t_value(0.0,net)
plot_t_value(0.25,net)
plot_t_value(0.50,net)
plot_t_value(0.75,net)